251.626

vns
 Athletes: ['james hall great', 'joshua nathan great', 'joe fraser great', 'jake jarman great']
Score: 256.993

hyper-heuristic
Athletes: ['jake jarman great' 'joe fraser great' 'james hall great'
 'max whitlock great']
 Score: 254.459

tabu search
Athletes:['joshua nathan great' 'joe fraser great' 'james hall great'
 'jake jarman great']
 Score:
256.993

In [19]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')


In [5]:
df=df[df['Nation']=='Britain']

In [7]:
df=df.drop(columns=['Rank', 'Nation', 'round', 'year', 'AA'])

In [8]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values
})

In [9]:
df = df_max_scores.copy()

vns
Selected Athletes: ['james hall great', 'joshua nathan great', 'joe fraser great', 'jake jarman great']
Total Score: 256.993

In [10]:
# VNS Algorithm with Iteration through random_state values
def variable_neighborhood_search(df):
    apparatus_order = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']  

    def calculate_total_score(athletes_df):
        # Sum the top 3 scores for each apparatus
        total_score = athletes_df[['FX', 'PH', 'SR', 'VT', 'PB', 'HB']].apply(lambda x: x.nlargest(3).sum()).sum()
        return total_score

    best_athletes = None
    best_score = float('-inf')
    best_seed = None

    for seed in range(6000): 
        # Randomly select 4 athletes with the current seed
        selected_athletes = df.sample(4, random_state=seed)

        # Check if each apparatus is competed on 3 times
        if all(selected_athletes[apparatus].count() >= 3 for apparatus in apparatus_order):
            # Calculate the total score for the current solution
            current_score = calculate_total_score(selected_athletes)

            # Compare with the current best solution
            if current_score > best_score:
                best_athletes = selected_athletes['Athlete'].tolist()
                best_score = current_score
                best_seed = seed

    return best_athletes, best_score, best_seed

# Run the VNS algorithm with different random_state values
best_athletes, total_score, best_seed = variable_neighborhood_search(df)

print("Selected Athletes:", best_athletes)
print("Total Score:", total_score)
print("Best Seed:", best_seed)


Selected Athletes: ['james hall great', 'joshua nathan great', 'joe fraser great', 'jake jarman great']
Total Score: 256.993
Best Seed: 517


hyper-heuristic

df = df_max_scores.copy()
Best Selection: ['jake jarman great' 'joe fraser great' 'james hall great'
 'max whitlock great']
Best Score: 254.459

In [12]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['jake jarman great' 'joe fraser great' 'james hall great'
 'max whitlock great']
Best Score: 254.459


tabu search
['joshua nathan great' 'joe fraser great' 'james hall great'
 'jake jarman great']
Total Score:
256.993

In [13]:
df = df_max_scores.copy()

In [14]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score



# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 213.09400000000002
Iteration 2, Best Score: 256.226
Iteration 3, Best Score: 256.993
Iteration 4, Best Score: 256.993
Iteration 5, Best Score: 256.993
Iteration 6, Best Score: 256.993
Iteration 7, Best Score: 256.993
Iteration 8, Best Score: 256.993
Iteration 9, Best Score: 256.993
Iteration 10, Best Score: 256.993
Iteration 11, Best Score: 256.993
Iteration 12, Best Score: 256.993
Iteration 13, Best Score: 256.993
Iteration 14, Best Score: 256.993
Iteration 15, Best Score: 256.993
Iteration 16, Best Score: 256.993
Iteration 17, Best Score: 256.993
Iteration 18, Best Score: 256.993
Iteration 19, Best Score: 256.993
Iteration 20, Best Score: 256.993
Iteration 21, Best Score: 256.993
Iteration 22, Best Score: 256.993
Iteration 23, Best Score: 256.993
Iteration 24, Best Score: 256.993
Iteration 25, Best Score: 256.993
Iteration 26, Best Score: 256.993
Iteration 27, Best Score: 256.993
Iteration 28, Best Score: 256.993
Iteration 29, Best Score: 256.993
Iteration 30

In [17]:
import pandas as pd

data = {
    'Method': ['vns', 'hyper-heuristic', 'tabu search'],
    'Athletes': [
        ['james hall great', 'joshua nathan great', 'joe fraser great', 'jake jarman great'],
        ['jake jarman great', 'joe fraser great', 'james hall great', 'max whitlock great'],
        ['joshua nathan great', 'joe fraser great', 'james hall great', 'jake jarman great']
    ],
    'Score': [256.993, 254.459, 256.993]
}

df = pd.DataFrame(data)
df


,Method,Athletes,Score
0,vns,"[james hall great, joshua nathan great, joe fr...",256.993
1,hyper-heuristic,"[jake jarman great, joe fraser great, james ha...",254.459
2,tabu search,"[joshua nathan great, joe fraser great, james ...",256.993


In [18]:
# df.to_csv('uk.csv', index=False)

In [20]:
df = pd.read_csv('../../../Combine_Data/men/iaa2024.csv')
df=df[df['Nation']=='Britain']
df=df.drop(columns=['Rank', 'Nation', 'round', 'year'])
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'PH': df.loc[idx_max['PH'], 'PH'].values,
    'SR': df.loc[idx_max['SR'], 'SR'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'PB': df.loc[idx_max['PB'], 'PB'].values,
    'HB': df.loc[idx_max['HB'], 'HB'].values,
    'AA': df.loc[idx_max['AA'], 'AA'].values

})


In [22]:
df=df_max_scores.copy()

In [23]:
df=df.sort_values(by='AA', ascending=False)

In [24]:
df

,Athlete,VT,PH,SR,FX,PB,HB,AA
6,jake jarman great,15.433,13.900,13.233,14.500,14.566,13.966,84.031
8,joe fraser great,14.266,13.166,13.933,13.900,15.066,14.266,83.964
7,james hall great,13.733,14.033,13.833,14.166,14.666,13.833,83.631
10,joshua nathan great,14.400,15.033,12.833,13.700,13.966,13.366,83.231
3,giarnni regini-moran great,14.633,13.433,0.000,14.533,14.966,7.133,64.698
1,courtney tulloch great,14.700,0.000,14.666,12.133,12.200,0.000,51.866
9,joseph cemlyn-jones great,0.000,12.666,13.100,0.000,9.933,12.633,48.332
4,harry hepworth great,14.466,0.000,14.366,14.500,0.000,0.000,43.332
11,max whitlock great,0.000,15.266,0.000,0.000,14.133,12.733,42.132
2,dominick cunningham great,14.633,0.000,0.000,11.466,0.000,0.000,26.099


In [25]:
84.031+83.964+83.631

251.626